In [1]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

print(shakespeare_text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [2]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize = "lower")

text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [3]:
encoded -= 2 # dropping tokens 0 (pad) and 1 (unknown), which won't be currently needed
n_tokens = text_vec_layer.vocabulary_size() - 2 #distinct token count
dataset_size = len(encoded)

In [4]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=12):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length+1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length+1))
    if shuffle:
        ds = ds.shuffle(buffer_size=100000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [5]:
tf.random.set_seed(42)

length = 100
train_set = to_dataset(encoded[:1000000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1000000:1060000], length=length)
test_set = to_dataset(encoded[1060000:], length=length)

In [6]:
'''model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model.keras", monitor="val_accuracy", save_best_only=True
)
history = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_ckpt])'''


'model = tf.keras.Sequential([\n    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),\n    tf.keras.layers.GRU(128, return_sequences=True),\n    tf.keras.layers.Dense(n_tokens, activation="softmax")\n])\nmodel.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])\nmodel_ckpt = tf.keras.callbacks.ModelCheckpoint(\n    "my_shakespeare_model.keras", monitor="val_accuracy", save_best_only=True\n)\nhistory = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_ckpt])'

In [7]:
model = tf.keras.models.load_model("my_shakespeare_model.keras")

#Wrapping the model with text vectorization and subtractring 2 from character IDs, since it cant do it on its own
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X-2),
    model
])


In [8]:
input_tensor = tf.convert_to_tensor(["To be or not to b"])
prediction = shakespeare_model.predict(input_tensor)
print(prediction)
y_proba = prediction[0][-1] #the probabilities for the last letter prediction - the one that we want to predict
print(y_proba)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 726ms/step
[[[1.92323610e-01 7.56669343e-02 1.09666749e-03 1.02835700e-01
   6.05005994e-02 5.16964383e-02 3.54065686e-01 4.74306894e-03
   4.56992947e-02 1.75471825e-04 1.02833863e-02 1.20571032e-02
   2.19551457e-05 1.23949926e-02 2.22976683e-04 1.38633391e-02
   4.23508231e-03 3.12037412e-02 2.26536024e-04 2.79061933e-04
   8.44501919e-05 3.89459601e-05 3.58245947e-04 4.50368319e-03
   2.41744288e-04 2.20014545e-06 1.16229225e-02 1.98218203e-03
   2.94302823e-03 3.50205111e-03 6.64717285e-04 4.53029905e-04
   8.72320561e-06 1.08449206e-06 1.16990111e-06 2.01257791e-07
   1.02548414e-09 2.75981440e-15 1.14279593e-12]
  [7.64866948e-01 1.89333281e-03 2.98201655e-07 5.73598482e-02
   1.98607049e-05 1.99654187e-05 2.98144215e-07 8.71299781e-05
   3.34221264e-03 5.25932536e-02 3.50211374e-02 5.09416917e-03
   3.31192382e-06 4.44670878e-02 1.81744695e-02 1.08137385e-06
   2.58129672e-04 4.45571495e-03 6.01200827e-06 1.85256031e-05
   5.23320341e-04 1.76423237e

In [9]:
y_pred = tf.argmax(y_proba)
print(text_vec_layer.get_vocabulary()[y_pred+2]) #adding 2, because since we subtracted it earlier, the get_vocabulary() method would return the wrong letter (one whose code is smaller by 2)

e


Testing categorical

In [10]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
print(tf.random.categorical(log_probas, num_samples=8))  # draw 8 samples

tf.Tensor([[0 1 0 2 1 0 0 1]], shape=(1, 8), dtype=int64)


In [11]:
def next_char(text, temperature=1):
    tensor_text = tf.convert_to_tensor([text])
    y_proba = shakespeare_model.predict(tensor_text, verbose=0)[0][-1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0][0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

def extended_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [12]:
print(extended_text("To be or not to b", temperature=0.01))
print(extended_text("To be or not to b", temperature=1))
print(extended_text("To be or not to b", temperature=10))

To be or not to be a profession.

provost:
i will not be so shall b
To be or not to bid for them dry from
the sense, and reason with hi
To be or not to b ysevje-cuo
dkzido'sie.p,iqeat;;.b ha?,;,?y? u-s.!


# Stateful RNN

In [ ]:
#previous dataset generation wouldnt work for a stateful RNN
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length+1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length+1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1000000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1000000:1060000], length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1060000:], length)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16, batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

#For resetting the state at the end of each epoch
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "shakespeare_model_stateful.keras", monitor="val_accuracy", save_best_only=True
)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set, epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

# Sentiment analysis

In [ ]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [ ]:
#Inspecting a couple of reviews
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8"))
    print("Label:", label.numpy())

In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))